In [1]:
import numpy as np
import os
import pickle
## for the original grover
def grover_unitary(nqubit,n):
    #famous -I+2A gate 
    inv = -np.eye(1<<nqubit,dtype=np.complex128)+2*np.ones((1<<nqubit,1<<nqubit),dtype=np.complex128)/(1<<nqubit)
    return [inv for i in range(n)]

def frac_oracle(state,index,alpha):
    state[index] *= np.exp(-1j*np.pi*alpha)
    return state

def apply_circuit(state,unitary_list,alpha,index):
    for i in range(len(unitary_list)-1):
        state = unitary_list[i]@state
        state = frac_oracle(state,index,alpha[i])
    state = unitary_list[-1]@state
    return state

def apply_circuit_all(state,unitary_list,alpha):
    success_rate = 0
    for i in range(state.shape[0]):
        tmp = np.copy(state)
        tmp = apply_circuit(tmp,unitary_list,alpha,i)
        success_rate += np.abs(tmp[i])**2
    return success_rate/state.shape[0]

In [2]:
folder = "unitary_data/"
files=os.listdir(folder)
for file in files:
    if not os.path.isdir(file):
        with open(folder+file,"rb") as f:
            data =pickle.load(f)
            num_qubit = data["num_qubit"]
            num_block = data["num_block"]
            unitary_list = data["unitary_list"]
            alpha = data["alpha"]
            state = np.zeros(2**num_qubit,dtype=np.complex128)
            state[0]=1
            success_rate = apply_circuit_all(state,unitary_list,alpha)
            print("%d-qubit\t%d-query\tsuccess rate:%f"%(num_qubit,num_block,success_rate))
            print("alpha:",alpha)

3-qubit	2-query	success rate:1.000000
alpha: [0.85496436 0.86275447]
4-qubit	3-query	success rate:1.000000
alpha: [0.98875366 0.99789003 0.97264992]
